In [78]:
import pandas as pd
import numpy as np

# Disable warnings from being printed
from warnings import filterwarnings
filterwarnings('ignore')

from sys import maxsize

from scipy.spatial.distance import pdist, squareform, euclidean

In [79]:
# Read the data
data = pd.read_csv("seeds_dataset.txt", sep=r"\s*", header=None)

# Given labels (natural clusters in data)
cluster_numbers = data.loc[:, 7].copy()
cluster_numbers_predicted = data.loc[:, 7].copy()
natural_clusters = cluster_numbers.unique().shape[0]

# Get attributes
data = data.loc[:, 1:6]

# Total number of points
n = data.shape[0]

# Total clusters formed in the end
total_clusters = 2*n - 1

# A hash of current unmerged clusters
current_clusters = np.array([1]*n + [0]*(n-1))

# A grid of distances between each pair of clusters
dist_grid = np.zeros((total_clusters, total_clusters))
initial_grid = squareform(pdist(data, 'euclidean'))
for i in range(n):
    for j in range(n):
        dist_grid[i][j] = initial_grid[i][j]
        
# To track all points in a cluster
cluster_points = []
for i in range(n):
    cluster_points.append((i, ))

In [80]:
# Total n-1 iterations
for i in range(n - 1):
               
    # Find minimum distance clusters
    mindist = maxsize
    minj = 0
    mink = 0
    for j in range(n + i):
        if current_clusters[j] == 0:
            continue
        for k in range(j + 1, n + i):
            if current_clusters[k] == 0:
                continue
            if mindist > dist_grid[j][k]:
                mindist = dist_grid[j][k]
                minj = j
                mink = k
               
    # Merge clusters
    cluster_points.append(cluster_points[minj] + cluster_points[mink])
    current_clusters[minj] = current_clusters[mink] = 0
    current_clusters[n + i] = 1
               
    # Update distances from other clusters
    centre = data.iloc[list(cluster_points[n + i])].mean().values
    for j in range(n + i):
        temp_centre = data.iloc[list(cluster_points[j])].mean().values
        dist_grid[n + i][j] = dist_grid[j][n + i] = np.linalg.norm(centre - temp_centre)
        
    # When current number of clusters equal natural clusters in data,
    # save the cluster for each point. This is used for calculating accuracy.
    if n - i + 1 == natural_clusters:
        for j in range(natural_clusters):
            for point in cluster_points[n + i - j]:
                cluster_numbers_predicted[point] = natural_clusters - j

In [81]:
# Map the original cluster labels to new cluster labels
mappings = {}
mappings_unavailable = []
for i in range(1, natural_clusters + 1):
    maxcnt = -1
    maxj = 0
    for j in range(1, natural_clusters + 1):
        if j in mappings_unavailable:
            continue
        # Count the number of points matching if i maps to j
        cnt = 0
        for k in range(n):
            if cluster_numbers[k] == i and cluster_numbers_predicted[k] == j:
                cnt = cnt + 1
        if maxcnt < cnt:
            maxcnt = cnt
            maxj = j
    mappings[i] = maxj
    mappings_unavailable.append(maxj)

for mapping in mappings.keys():
    cluster_numbers[cluster_numbers == mapping] = mappings[mapping]

In [82]:
# Finally compute accuracy
cnt = 0.0
for i in range(n):
    if cluster_numbers[i] == cluster_numbers_predicted[i]:
        cnt = cnt + 1.0
print("Accuracy: ", cnt/n)

Accuracy:  0.6428571428571429
